In [28]:
import tensorflow as tf
import numpy as np

Vi starter med at loaded den givne data ind. Her er dataen sepereret i lyd med musik og lyd uden musik. Det skal vi så sætte sammen i et array og lave en array som indeholder om hvilken type hver styk data er. Her kan vi bruge numpy siden hver fil er i .npy format

In [29]:
music_data = np.load("music_data.npy")
other_data = np.load("other_data.npy")
unknown_data = np.load("test_data.npy")

Data layoutet er her $10500$ 2d arrays af $30\times79$ som repræsentere et log-mel spektogram udregnet fra den givne lyd sample. $x$ dimensionen er frekvens båndene mens $y$ er tids rammen

Nu skal vi sætte `music_data` og `other_data` sammen til et array og lave en array som indeholder hvilken type hver lyd sample er. Vi vil også gerne have noget test data så vi kan evaluere hvor godt netværket er uden at overfitte Det gør vi ved at efterlade 1000 fra `music_dat` og `other_data`

In [30]:
music_train_data = music_data[0: -1000]
other_train_data = other_data[0: -1000]
train_data = (
	np.concatenate((music_train_data, other_train_data)),
	np.concatenate((
		np.full((len(music_train_data), 2), (1, 0)), 
		np.full((len(other_train_data), 2), (0, 1))
	))
)	

music_test_data = music_data[len(music_data) - 1000:None]
other_test_data = other_data[len(other_data) - 1000:None]
test_data = (
	np.concatenate((music_test_data, other_test_data)),
	np.concatenate((
		np.full((len(music_test_data), 2), (1, 0)),
		np.full((len(other_test_data), 2), (0, 1))
	))
)


Nu har vi lavet dataset til test og træning, nu skal vi sætte vores model op

In [31]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Reshape((30, 79, 1), input_shape = (30, 79)),
	tf.keras.layers.Conv2D(3, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(),
	tf.keras.layers.Conv2D(32, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(),
	tf.keras.layers.Conv2D(64, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(),
	tf.keras.layers.Flatten(),
	#tf.keras.layers.Dense(1024, activation = "relu"),
	#tf.keras.layers.Dense(512, activation = "relu"),
	tf.keras.layers.Dense(2, activation = "softmax")
])

model.build(input_shape=(30, 79))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 30, 79, 1)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 77, 3)         30        
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 38, 3)        0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 12, 36, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 6, 18, 32)        0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 16, 64)        

Nu hvor vi har en model, så skal den bare trænes

In [32]:
predictions = model(train_data[0][:10]).numpy()
predictions

array([[7.7406534e-05, 9.9992263e-01],
       [8.0589205e-04, 9.9919409e-01],
       [4.1797823e-05, 9.9995816e-01],
       [2.8487327e-04, 9.9971515e-01],
       [7.5976911e-04, 9.9924028e-01],
       [3.3495733e-05, 9.9996650e-01],
       [3.3407502e-03, 9.9665928e-01],
       [1.1044008e-03, 9.9889553e-01],
       [4.2465082e-03, 9.9575353e-01],
       [2.0498382e-03, 9.9795014e-01]], dtype=float32)

Nu laver vi en loss funktion, siden vi kun har et output så bruger vi binarycrossentropy

In [33]:
model.compile(optimizer='adam', loss='CategoricalCrossentropy', metrics=['accuracy'])

Så træner vi modellen med dataen over 10 epochs

In [34]:
history = model.fit(train_data[0], train_data[1], epochs=10)

Epoch 1/10
594/594 [==============================] - 10s 15ms/step - loss: 0.3708 - accuracy: 0.8547
Epoch 2/10
594/594 [==============================] - 9s 15ms/step - loss: 0.1517 - accuracy: 0.9433
Epoch 3/10
594/594 [==============================] - 9s 15ms/step - loss: 0.1003 - accuracy: 0.9650
Epoch 4/10
155/594 [======>.......................] - ETA: 6s - loss: 0.0895 - accuracy: 0.9681

In [ ]:
train_evaluation = model.evaluate(train_data[0], train_data[1])
test_evaluation = model.evaluate(test_data[0], test_data[1])
print("Training set accuracy %.3f%% and loss: %.3f%%" % 
	(train_evaluation[1] * 100, train_evaluation[0] * 100))
print("Test set accuracy %.3f%% and loss: %.3f%%" % 
	(test_evaluation[1] * 100, test_evaluation[0] * 100))

63/63 [==============================] - 0s 7ms/step - loss: 0.0555 - accuracy: 0.9820
Training set accuracy 99.237% and loss: 2.466%
Test set accuracy 98.200% and loss: 5.551%


In [ ]:
data = model(unknown_data).numpy()

In [ ]:
f = open("predictions.txt", "w")

first = True

for prediction in data:
	if first:
		first = False
	else:
		f.write("\n")
	if prediction[0] > prediction[1]:
		f.write("1")
	else:
		f.write("0")
f.close()

For lige at være sikker på modellen giver det vi forventer, så printer vi det lige her

In [ ]:
pred = model(music_data[:1]).numpy()[0]
print("%.3f %.3f" % (pred[0], pred[1]))
pred = model(other_data[:1]).numpy()[0]
print("%.3f %.3f" % (pred[0], pred[1]))

0.999 0.001
